In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [10]:
studio_id = 3

studio_scenes = stash_client.find_scenes_by_studio(studio_id)
studio_galleries = stash_client.find_galleries_by_studio(studio_id)


In [11]:
studio_scenes_without_galleries = studio_scenes.filter(pl.col("stashapp_galleries").list.len() == 0).with_columns(pl.col("stashapp_title").str.to_lowercase().str.replace(",", "").str.replace("?", "").alias("stashapp_title_cleaned"))
studio_galleries_without_scenes = studio_galleries.filter(pl.col("stashapp_scenes").list.len() == 0).with_columns(pl.col("stashapp_title").str.to_lowercase().str.replace(",", "").str.replace("?", "").alias("stashapp_title_cleaned"))

In [12]:
excluded_scene_stashapp_ids = []

In [ ]:
joined_scenes_to_galleries = studio_scenes_without_galleries.with_columns().join(studio_galleries_without_scenes, on="stashapp_title_cleaned", how="left", coalesce=False, suffix="_gallery").select(
    pl.col("stashapp_id"),
    pl.col("stashapp_title"),
    pl.col("stashapp_date"),
    pl.col("stashapp_performers").map_elements(lambda x: [performer.get("stashapp_performers_id") for performer in x], return_dtype=pl.List(pl.Int64)).alias("stashapp_performers_ids"),
    pl.col("stashapp_performers").map_elements(lambda x: [performer.get("stashapp_performers_name") for performer in x], return_dtype=pl.List(pl.Utf8)).alias("stashapp_performer_names"),
    pl.col("stashapp_id_gallery"),
    pl.col("stashapp_title_gallery"),
    pl.col("stashapp_date_gallery"),
    pl.col("stashapp_performers_gallery").map_elements(lambda x: [performer.get("stashapp_performers_id") for performer in x], return_dtype=pl.List(pl.Int64)).alias("stashapp_performers_ids_gallery"),
    pl.col("stashapp_performers_gallery").map_elements(lambda x: [performer.get("stashapp_performers_name") for performer in x], return_dtype=pl.List(pl.Utf8)).alias("stashapp_performers_names_gallery"),
).filter(
    pl.col("stashapp_id_gallery").is_not_null(),
    ~pl.col("stashapp_id").is_in(excluded_scene_stashapp_ids)
)
current_batch = joined_scenes_to_galleries.head(10)
current_batch


In [ ]:
current_batch_update_galleries_inputs = []
for row in current_batch.iter_rows(named=True):
    current_batch_update_galleries_inputs.append({
        "id": row["stashapp_id_gallery"],
        "title": row["stashapp_title"],
        "scene_ids": [row["stashapp_id"]],
        "performer_ids": row["stashapp_performers_ids"],
    })
current_batch_update_galleries_inputs


In [16]:
for update_gallery_input in current_batch_update_galleries_inputs:
    stash.update_gallery(update_gallery_input)